In [2]:
# 코랩에서 mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-12-06 09:48:40--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-12-06 09:48:40--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc111a1c5087260dd9f903e483b.dl.dropboxusercontent.com/cd/0/get/ByHqrIxudw8DTgrIAu_PSjQxdWAtQ1etRt68BMS_LPQYZGnbK3bPqhY2mNOuAaZ8RxBDjAkeVVaHogGso3ha1KfPCLYz1VpfnTOV0iyuhuUAGlxXTsmX5_1xnNOlfxlM5y7ASyN5uD7NkA7c4k0Th9vbZF8uk4NUrVFDvwnIjfyULVdFxFMZfbYz9AaojXQIFfs/file?dl=1# [following]
--2022-12-06 09:48:41--  https://ucc111a1c5087260dd9f903e483b.dl.dropboxusercontent.com/cd/0/get/ByHqrIxudw8D

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
import numpy as np

In [4]:

data = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/1.prep_data_dotsplits.csv')
data

,question,answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자세한 설명해주시면 ...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,1년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요...,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,연고도 다르고 일면식도 없는 5살위의 형을 일을 통해 알게 됐습니다.제가 주식투자를...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,1 21615 이행권고결정판결 2 211523 채권압류 및 추심명령 인용 21153...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,4천만 원 정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사 중...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18160,제가 작년 8월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로...,통장 압류 방법에 관하여 질문하셨습니다. 민사소송에서 승소를 하여 그 판결이 확정된...
18161,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다. 나와서 따로 혼자 사는데 ...,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18162,추완항소이고 제가 피고인이예요 공시송달로 소송을 모르고 있다가 처음부터 끝까지 공시...,민사소송 항소이유를 잘못 기재한 것과 관련하여 질문하셨습니다. 부동산 임대차에 있어...
18163,작년 여름 사기 피해 금액 약 35만 원을 당하여 형사고소를 하였고 1월경 형사 고...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [9]:
# 다음 뉴스 본문
df = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/society_daum_news.csv')
df = df.drop_duplicates('URL')
df = df.drop_duplicates('Content')
df = df.dropna(subset=['Content'])
df = df.reset_index(drop=True)
df

,Unnamed: 0,Content,DomainID,MainCategory,PhotoURL,Press,Stickers,SubCategory,Title,URL,WritedAt,Writer
0,0,"국회 행안위 소위 25억 삭감,전자충격기, 순찰차 유류비도 줄줄이 깎여[아시아경제 ...",0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,아시아경제,"{'article_id': '20221130104657051', '좋아요': 133...",사건/사고,尹대통령 주문한 ‘1인 1총기’ 예산 25억·전자충격기 전액 삭감,https://v.daum.net/v/20221130104657051,2022. 11. 30. 10:46,장세희
1,1,(부산=연합뉴스) 손형주 기자 = 음주 측정을 거부하며 경찰관 3명을 폭행한 20대...,0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,연합뉴스,"{'article_id': '20221130105301310', '좋아요': 0, ...",사건/사고,음주 측정 거부하고 경찰관 3명 폭행한 20대 검거,https://v.daum.net/v/20221130105301310,2022. 11. 30. 10:53,손형주
2,2,이태원역장도 오늘 소환(서울=뉴스1) 손승환 기자 = 이태원 참사를 수사 중인 경찰...,0,사회,https://img1.daumcdn.net/thumb/R658x0.q70/?fna...,뉴스1,"{'article_id': '20221130105309315', '좋아요': 0, ...",사건/사고,"특수본 출석 용산구청 부청장 ""죄송하다…성실히 수사 임할 것""",https://v.daum.net/v/20221130105309315,2022. 11. 30. 10:53,손승환 기자
3,3,국민의힘과 더불어민주당은 지난 23일 이태원 참사 국정조사를 합의했다. 24일부터 ...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,더팩트,"{'article_id': '20221125000025485', '좋아요': 1, ...",사건/사고,"이태원 참사 국정조사, 특수본 수사에 '약'일까 '독'일까",https://v.daum.net/v/20221125000025485,2022. 11. 25. 00:00,최의종
4,4,"서산서 60대 엽사, 동료 총에 숨져,경찰, 업무상과실치사 혐의로 입건,최근 포상금...",0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,동아일보,"{'article_id': '20221114030332296', '좋아요': 2, ...",사건/사고,"또 멧돼지로 오인사격, 올해만 3번째 사망자",https://v.daum.net/v/20221114030332296,2022. 11. 14. 03:03,서산=이기진 기자
...,...,...,...,...,...,...,...,...,...,...,...,...
54027,58047,대전지검 천안지청은 28일 박상돈 천안시장에 대해 지난 6.1 지방선거당시 공무원 ...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,더팩트,"{'article_id': '20221129110734310', '좋아요': 0, ...",전국,천안·아산 시장 ‘공직선거법 위반’혐의로 재판 받는다,https://v.daum.net/v/20221129110734310,2022. 11. 29. 11:07,김경동
54028,58048,벡스코는 28일 부산 서구노인복지관에 지역특산품 200세트(300만 원 상당)를 기...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110004878', '좋아요': 0, ...",전국,"벡스코, 노인복지관에 지역특산품 200세트 기부",https://v.daum.net/v/20221129110004878,2022. 11. 29. 11:00,부산CBS 박상희 기자
54029,58049,30일(수) 오전 0시 '동파 주의(2단계)' 발령30일부터 서울 전역에 아침 최저...,0,사회,https://img3.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110008889', '좋아요': 0, ...",전국,30일 최저 영하7도…서울 수도계량기 '동파주의' 발령,https://v.daum.net/v/20221129110008889,2022. 11. 29. 11:00,CBS노컷뉴스 김민수 기자
54030,58050,"내일 아침 체감온도 영하 15도…""등굣길·출근길 추위 대비"",이번 추위, 다음 달 ...",0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110309117', '좋아요': 0, ...",전국,"충남권 한파특보…오늘 밤부터 영하권 ""퇴근길 추워요""",https://v.daum.net/v/20221129110309117,2022. 11. 29. 11:03,대전CBS 김미성 기자


In [10]:
# # 다음 뉴스 본문 전문
# df = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/news_2.csv')
# df = df.drop_duplicates('URL')
# df = df.drop_duplicates('Content')
# df = df.dropna(subset=['Content'])
# df = df.reset_index(drop=True)
# df

In [11]:
def preprocessing(idx):
    content = df.loc[idx, 'Content']
    # 숫자 한글 영문 , . 공백을 제외한 문자 제거
    content_pre = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣. ]', '', content)
    # 소수 및 정수 제거
    content_pre = re.sub('\d+.\d*', '',content_pre)
    # url 및 이메일 제거
    content_pre = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content_pre)
    # 다수의 점 (ex : ...) 점 한개로 대체
    content_pre = re.sub("\.+\.", '.', content_pre)
    # 다수의 공백 축소
    content_pre = re.sub(' +', ' ', content_pre)

    # 기자명, 언론사명 stopword 처리
    wrt = df.loc[idx, 'Writer']
    press = df.loc[idx, 'Press']

    # stopword 추가해서 처리
    stop_word = [f'{wrt}', f'{press}', '기자', 'Copyrights',  '무단 전재 및 재배포 금지']
    content_pre2 = content_pre
    for stp in stop_word :
        content_pre2 = content_pre2.replace(stp, '')
    return content_pre2

In [12]:
import re
df_content = pd.DataFrame(columns=['content'])
for idx in tqdm(df.index):
    temp_content = preprocessing(idx)
    temp_df = pd.DataFrame({'content' : temp_content}, index=[0])
    df_content = pd.concat([df_content, temp_df])
df_content = df_content.reset_index(drop=True)
df_content

  0%|          | 0/54032 [00:00<?, ?it/s]

,content
0,국회 행안위 소위 삭감전자충격기 순찰차 유류비도 줄줄이 깎여 윤석열 대통령의 지시...
1,부산 음주 측정을 거부하며 경찰관 을 폭행한 남성이 경찰에 붙잡혔다. 부산 남부...
2,이태원역장도 오늘 소환서울뉴스 이태원 참사를 수사 중인 경찰청 특별수사본부특수본가 ...
3,국민의힘과 더불어민주당은 지난 이태원 참사 국정조사를 합의했다. 부터 간 진행하되 ...
4,서산서 엽사 동료 총에 숨져경찰 업무상과실치사 혐의로 입건최근 포상금 인상 포획경쟁...
...,...
54027,대전지검 천안지청은 박상돈 천안시장에 대해 지난 지방선거당시 공무원 조직을 이용해 ...
54028,벡스코는 부산 서구노인복지관에 지역특산품 트 원 상당를 기부했다.벡스코는 지난 열린...
54029,수 오전 동파 주의계 발령부터 서울 전역에 아침 최저 영하 의 강추위가 예보된 가운...
54030,내일 아침 체감온도 영하 등굣길출근길 추위 대비이번 추위 다음 달 까지 지속될 듯 ...


In [13]:
def get_df_token(data):

    mecab = Mecab()
    q_pos = []

    for i in tqdm(range(len(data))):
        q_pos.append(mecab.pos(data['content'][i]))
    data['content_token'] = q_pos

    pos = ["NNG","NNP",   # 명사
         "VV","VA"   # 동사
         ]

    data['content_token'] = data['content_token'].apply(lambda x: [t for (t, p) in x if p in pos])

    return data

def get_ques_token(content_token):
    mecab = Mecab()
    ques_token = mecab.pos(content_token)
    ques_token = ques_remove(ques_token)

    return ques_token

def ques_remove(content_token) :

    pos = ["NNG","NNP",   # 명사
           "VV","VA"   # 동사
           ]

    ques_token = [[t for (t, p) in content_token if p in pos]]

    return ques_token

def jaccard_similarity(data, content_token):

    union = set(data).union(set(content_token))
    intersection = set(data).intersection(set(content_token))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, content_token, num):
    
    # data: 데이터프레임, content_token: 입력한 텍스트(content_token), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data['content_token'].apply(lambda x: jaccard_similarity(x, content_token[0]))

    return data[['content', 'content_token', 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, content_token):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data['content_token'])
    tfidf_content_token = tfidf_vectorizer.transform(content_token)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_content_token).reshape(-1, )

    return data[['content', 'content_token', 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def cos_high(data):
    if data['cosine_similarity'][0] >= 0.6:
        return data['content'][0]

    else:
        return print('모델구현')


In [14]:
# 함수 튜닝
def get_df_token(data, input_col, output_col):

    mecab = Mecab()
    q_pos = []

    for i in tqdm(range(len(data))):
        q_pos.append(mecab.pos(data[input_col][i]))
    data[output_col] = q_pos

    pos = ["NNG","NNP",   # 명사
         "VV","VA"   # 동사
         ]

    data[output_col] = data[output_col].apply(lambda x: [t for (t, p) in x if p in pos])

    return data

def get_ques_token(content_token):
    mecab = Mecab()
    ques_token = mecab.pos(content_token)
    ques_token = ques_remove(ques_token)

    return ques_token

def ques_remove(content_token) :

    pos = ["NNG","NNP",   # 명사
           "VV","VA"   # 동사
           ]

    ques_token = [[t for (t, p) in content_token if p in pos]]

    return ques_token

def jaccard_similarity(data, content_token):

    union = set(data).union(set(content_token))
    intersection = set(data).intersection(set(content_token))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, content_token, num, input_col, output_col):
    
    # data: 데이터프레임, content_token: 입력한 텍스트(content_token), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data[output_col].apply(lambda x: jaccard_similarity(x, content_token[0]))

    return data[[input_col, output_col, 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, content_token, input_col, output_col):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data[output_col])
    tfidf_content_token = tfidf_vectorizer.transform(content_token)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_content_token).reshape(-1, )

    return data[[input_col, output_col, 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def cos_high(data, input_col, output_col):
    if data['cosine_similarity'][0] >= 0.6:
        return data[input_col][0]

    else:
        return print('모델구현')


In [15]:
mecab = Mecab()
q_pos = []

for i in tqdm(range(len(df_content))):
    q_pos.append(mecab.pos(df_content['content'][i]))
df_content['content_token'] = q_pos

  0%|          | 0/54032 [00:00<?, ?it/s]

In [16]:
df_content['content_token']

0        [(국회, NNG), (행안, NNP), (위, NNG), (소위, NNG), (삭...
1        [(부산, NNP), (음주, NNG), (측정, NNG), (을, JKO), (거...
2        [(이태원, NNP), (역장, NNG), (도, JX), (오늘, MAG), (소...
3        [(국민, NNG), (의, JKG), (힘, NNG), (과, JKB), (더불,...
4        [(서산, NNP), (서, JKB), (엽사, NNG), (동료, NNG), (총...
                               ...                        
54027    [(대전, NNP), (지검, NNG), (천안, NNP), (지청, NNG), (...
54028    [(벡스코, NNP), (는, JX), (부산, NNP), (서구, NNG), (노...
54029    [(수, NNG), (오전, NNG), (동파, NNG), (주의, NNG), (계...
54030    [(내일, MAG), (아침, NNG), (체감, NNG), (온도, NNG), (...
54031    [(부산, NNP), (에서, JKB), (하루, NNG), (확진, NNG), (...
Name: content_token, Length: 54032, dtype: object

In [17]:
content1 = df_content.iloc[0, 0]
content1

'국회 행안위 소위 삭감전자충격기 순찰차 유류비도 줄줄이 깎여  윤석열 대통령의 지시사항이었던 기 소지 예산이 국회에서 대폭 삭감된 것으로 확인됐다. 위급한 상황에서 상대를 제압할 수 있는 전자충격기 예산도 전액 삭감됐다.  취재를 종합하면 최근 국회 행정안전위원회 소위원회에서 기 소지 예산은 만원이 삭감됐다. 전체 만원의 달하는 수준이다. 예산 기준 권총 한 개당 단가는 원인 것을 고려했을 때 개의 권총을 추가로 구매할 수 없게 된 것이다. 더불어민주당 측은 경 리볼버 총기의 경우 살상력이 높아 과잉진압과 인권침해 피소 가능성 등의 문제로 효용성에 한계가 있다고 지적했다. 행안위 전문위원실 역시 인권침해와 피소 가능성 등을 문제 삼으며 예산 확대의 긴급성이 인정되지 않는다고 봤다.경찰청 관계자는 에 몇 번 안 되지만 권총을 꼭 사용해야 하는 경우가 있다며 현직 경찰이 국민과 자신의 생명을 지키기 위해서는 권총을 사용해야 하는 순간이 있다고 설명했다. 경찰 복수의 관계자들 역시 경찰 권총 예산을 삭감하는 것은 소방관에게 소화기를 지급하지 않는 것과 같다고 주장한다. 앞서 경찰은 윤 대통령이 지난 서울 신촌지구대를 방문해 흉악범 대응 방안으로 기 소지를 검토하라고 지시하자 올해 예산만원 보다 대폭 상향해 예산을 편성했다. 당초 편성한 예산의 절반 이상이 삭감되면서 대통령 지시사항 이행에도 차질을 빚을 것이란 전망이 나온다.상대를 제압할 때 쓰는 전자충격기 예산은 만원 전액 삭감됐다. 개발됐음에도 달하는 불량률로 상용화되지 못했다는 게 이유다. 국민의힘과 민주당은 예산 심사 당시 모두 전자충격기의 안정성을 입증이 선행돼야 한다고 강조한 것으로 전해졌다. 이외에 경찰차 유류비 예산도 원 감액된 것으로 확인됐다. 유류비 시세 변동 폭이 반영된 결과로 풀이된다.현장에서는 권총 배치 실효성에 대한 의문은 여전히 남아있다. 서울지역 지구대에서 근무하는 한 팀장은 권총을 배치하는 것에 초점을 맞추기보다는 책임에 대한 부분을 분명히 해주면 좋을 것 같다며 모든 책임이 현장 일

In [18]:
df_content

,content,content_token
0,국회 행안위 소위 삭감전자충격기 순찰차 유류비도 줄줄이 깎여 윤석열 대통령의 지시...,"[(국회, NNG), (행안, NNP), (위, NNG), (소위, NNG), (삭..."
1,부산 음주 측정을 거부하며 경찰관 을 폭행한 남성이 경찰에 붙잡혔다. 부산 남부...,"[(부산, NNP), (음주, NNG), (측정, NNG), (을, JKO), (거..."
2,이태원역장도 오늘 소환서울뉴스 이태원 참사를 수사 중인 경찰청 특별수사본부특수본가 ...,"[(이태원, NNP), (역장, NNG), (도, JX), (오늘, MAG), (소..."
3,국민의힘과 더불어민주당은 지난 이태원 참사 국정조사를 합의했다. 부터 간 진행하되 ...,"[(국민, NNG), (의, JKG), (힘, NNG), (과, JKB), (더불,..."
4,서산서 엽사 동료 총에 숨져경찰 업무상과실치사 혐의로 입건최근 포상금 인상 포획경쟁...,"[(서산, NNP), (서, JKB), (엽사, NNG), (동료, NNG), (총..."
...,...,...
54027,대전지검 천안지청은 박상돈 천안시장에 대해 지난 지방선거당시 공무원 조직을 이용해 ...,"[(대전, NNP), (지검, NNG), (천안, NNP), (지청, NNG), (..."
54028,벡스코는 부산 서구노인복지관에 지역특산품 트 원 상당를 기부했다.벡스코는 지난 열린...,"[(벡스코, NNP), (는, JX), (부산, NNP), (서구, NNG), (노..."
54029,수 오전 동파 주의계 발령부터 서울 전역에 아침 최저 영하 의 강추위가 예보된 가운...,"[(수, NNG), (오전, NNG), (동파, NNG), (주의, NNG), (계..."
54030,내일 아침 체감온도 영하 등굣길출근길 추위 대비이번 추위 다음 달 까지 지속될 듯 ...,"[(내일, MAG), (아침, NNG), (체감, NNG), (온도, NNG), (..."


In [19]:
df_content = get_df_token(df_content, 'content', 'content_token')
df_content

  0%|          | 0/54032 [00:00<?, ?it/s]

,content,content_token
0,국회 행안위 소위 삭감전자충격기 순찰차 유류비도 줄줄이 깎여 윤석열 대통령의 지시...,"[국회, 행안, 위, 소위, 삭감, 전자, 충격, 기, 순찰차, 유류, 비, 윤석열..."
1,부산 음주 측정을 거부하며 경찰관 을 폭행한 남성이 경찰에 붙잡혔다. 부산 남부...,"[부산, 음주, 측정, 거부, 경찰관, 폭행, 남성, 경찰, 부산, 남부, 경찰서,..."
2,이태원역장도 오늘 소환서울뉴스 이태원 참사를 수사 중인 경찰청 특별수사본부특수본가 ...,"[이태원, 역장, 소환, 서울, 뉴스, 이태원, 참사, 수사, 경찰청, 특별, 수사..."
3,국민의힘과 더불어민주당은 지난 이태원 참사 국정조사를 합의했다. 부터 간 진행하되 ...,"[국민, 힘, 더불, 민주당, 이태원, 참사, 국정, 조사, 합의, 부, 터, 진행..."
4,서산서 엽사 동료 총에 숨져경찰 업무상과실치사 혐의로 입건최근 포상금 인상 포획경쟁...,"[서산, 엽사, 동료, 총, 경찰, 업무상, 과실, 치사, 혐의, 입건, 최근, 포..."
...,...,...
54027,대전지검 천안지청은 박상돈 천안시장에 대해 지난 지방선거당시 공무원 조직을 이용해 ...,"[대전, 지검, 천안, 지청, 박상돈, 천안시장, 지방, 선거, 당시, 공무원, 조..."
54028,벡스코는 부산 서구노인복지관에 지역특산품 트 원 상당를 기부했다.벡스코는 지난 열린...,"[벡스코, 부산, 서구, 노인, 복지, 관, 지역, 특산품, 트, 원, 상당, 기부..."
54029,수 오전 동파 주의계 발령부터 서울 전역에 아침 최저 영하 의 강추위가 예보된 가운...,"[수, 오전, 동파, 주의, 발령, 서울, 전역, 아침, 최저, 영하, 강추위, 예..."
54030,내일 아침 체감온도 영하 등굣길출근길 추위 대비이번 추위 다음 달 까지 지속될 듯 ...,"[아침, 체감, 온도, 영하, 등굣길, 출근길, 추위, 대비, 이번, 추위, 다음,..."


In [20]:
ques_token = get_ques_token(content1)
ques_token

[['국회',
  '행안',
  '위',
  '소위',
  '삭감',
  '전자',
  '충격',
  '기',
  '순찰차',
  '유류',
  '비',
  '윤석열',
  '대통령',
  '지시',
  '사항',
  '기',
  '소지',
  '예산',
  '국회',
  '삭감',
  '확인',
  '상황',
  '상대',
  '제압',
  '있',
  '전자',
  '충격',
  '기',
  '예산',
  '전액',
  '삭감',
  '취재',
  '종합',
  '최근',
  '국회',
  '행정',
  '안전',
  '위원회',
  '소위원회',
  '기',
  '소지',
  '예산',
  '삭감',
  '전체',
  '달',
  '수준',
  '예산',
  '기준',
  '권총',
  '개당',
  '단가',
  '원',
  '고려',
  '때',
  '권총',
  '추가',
  '구매',
  '없',
  '더불',
  '민주당',
  '경',
  '리볼버',
  '총기',
  '경우',
  '살상력',
  '높',
  '과잉',
  '진압',
  '인권',
  '침해',
  '피소',
  '가능',
  '문제',
  '효용',
  '한계',
  '있',
  '지적',
  '행안',
  '위',
  '전문',
  '위원',
  '실',
  '인권',
  '침해',
  '피소',
  '가능',
  '문제',
  '삼',
  '예산',
  '확대',
  '긴급',
  '인정',
  '경찰청',
  '관계자',
  '되',
  '권총',
  '사용',
  '하',
  '경우',
  '있',
  '현직',
  '경찰',
  '국민',
  '자신',
  '생명',
  '지키',
  '권총',
  '사용',
  '하',
  '순간',
  '있',
  '설명',
  '경찰',
  '복수',
  '관계자',
  '경찰',
  '권총',
  '예산',
  '삭감',
  '소방관',
  '소화기',
  '지급',
  '같',
  '주장',
  '경찰',
  '윤',
  

In [21]:
result = jaccard_high(df_content, ques_token, 100, 'content', 'content_token')
result = cos_similarity(result, ques_token, 'content', 'content_token')

In [22]:
result

,content,content_token,jaccard_similarity,cosine_similarity
0,국회 행안위 소위 삭감전자충격기 순찰차 유류비도 줄줄이 깎여 윤석열 대통령의 지시...,"[국회, 행안, 위, 소위, 삭감, 전자, 충격, 기, 순찰차, 유류, 비, 윤석열...",1.000000,1.000000
124,흉기로 저항하던 남성에 실탄을 쏴 검거한 경찰 대응에 대해 전문가는 적절했다...,"[흉기, 저항, 남성, 실탄, 검거, 경찰, 대응, 전문가, 평가, 부산, 사상, ...",0.112628,0.204043
31453,국방위원회 전체소위 회의록으로 살펴 본 이전 예산 삭감 과정국방위 민주당 의원 반대...,"[국방위원회, 전체, 소위, 회의록, 이전, 예산, 삭감, 과정, 국방위, 민주당,...",0.119792,0.182006
12820,국방위 원만 예산 반영대전시 국토 균형발전 저해 우려예결위 소위서 복구 요청할 것방...,"[국방위, 원만, 예산, 반영, 대전시, 국토, 균형발전, 저해, 우려, 예결, 위...",0.125954,0.169677
36776,지자체 지역 정치권 언론 등 한 목소리 낸 끝에 약 만에 복원예결특위 국방위 동의 ...,"[지자체, 지역, 정치, 언론, 목소리, 끝, 약, 복원, 예결, 특위, 국방위, ...",0.116041,0.149446
...,...,...,...,...
9827,병무청장 심각하게 생각하고 있다서울 일부 대학교 학생들이 예비군 훈련 참석으로 ...,"[병무, 청장, 생각, 서울, 일부, 대학교, 학생, 예비군, 훈련, 참석, 이익,...",0.103321,0.028164
47356,대구시와 대구시교육청이 이견을 보이는 부분은 지. 위반건수에 대한 입장과 보조금 집...,"[대구시, 대구시, 교육청, 이견, 보이, 부분, 지, 위반, 건수, 입장, 보조금...",0.102389,0.025403
8941,전문병원 응급실 운영에 정부대구시 지원 절실특히 전문병원 응급실 운영에 정부와 대구...,"[전문, 병원, 응급실, 운영, 정부, 대구시, 지원, 전문, 병원, 응급실, 운영...",0.102679,0.024655
46212,특정 은행에 유리한 평가지표 지표 내용 및 배점 구성 문제 심각마이너스 적용 가산금...,"[특정, 은행, 유리, 평가, 지표, 지표, 내용, 배점, 구성, 문제, 마이너스,...",0.106383,0.024276


# 파생변수 생성
1. 본문 유사도 50% : 자카드 / 코사인 비율 2:1. 
-> 유사도의 경우 가장 근접한 수치 기준으로 MinMaxScaling
-> 유사도의 비중을 더욱 높게 반영하고 연산속도를 위해 유사도 기준 상위 100개에 
2. 본문 핵심문장 유사도 30% : 본문과 산정방식 동일. / TextRank 같은 걸로 한 1~2문장정도 요약해서 유사도를 해볼까.
3. 같은 메인카테고리 주제에 대해선 10% 가중치
4. 같은 서브카테고리 주제에 대해선 5% 가중치
5. 1일 근접한 기사에 대해서는 1, 3일내 0.5, 7일내 0.3, 이외의 기사 가중치 0, NaN값은 0으로 처리. 전체 가중치 지분 5% 정도
<!-- 6. 같은 언론사의 기사에 대해서는 5% 가중치 -->

In [ ]:
import networkx as nx

In [ ]:
import networkx as nx
new_lines = []
dictCount={}
tag_sen = []
dictBicount={}

ratio = 0.666
max_sen = 5
threshold = 0.005
def similarity(a, b):
    try:
        n = len(a.intersection(b))
        return n / float(len(a) + len(b) - n)
    except:
        return 0
def build_dictCount(sentences):
    for sent in sentences.split('.'):
        temp_lst = []
        temp = Mecab().pos(sent)
        for i in temp:
            if i[1] in ('NNG', 'NNP', 'VV', 'VA'):
                temp_lst.append(i)
        tag_sen.append(set(temp_lst))
        dictCount[len(dictCount)] = sent
    return dictCount, tag_sen

def build_dictBiCount(dictCount, tag_sen):
    for i in range(len(dictCount)):
        for j in range(i+1, len(dictCount)):
            s = similarity(tag_sen[i], tag_sen[j])
            if s < threshold:
                continue
            dictBicount[i, j] = s
    return dictBicount

def build(dictCount, dictBicount):
    graph = nx.Graph()
    graph.add_nodes_from(dictCount.keys())
    for (a, b), n in dictBicount.items():
        graph.add_edge(a, b, weight=n*1.0 + (1-1.0))
    return graph

def pagerank(graph):
    rank = nx.pagerank(graph, weight='weight')
    return rank


def new_line(rank, dictCount):
    ks = sorted(rank, key=rank.get, reverse=True)
    score = int(len(rank)*ratio)

    if score < max_sen :
        score = int(len(rank)*ratio)
    elif score >= max_sen:
        score = max_sen
    else:
        pass

    if score == 0:
        score = len(rank)

    ks = ks[:score]
    new_text = '.'.join(map(lambda k: dictCount[k], sorted(ks)))
    return new_text 

In [140]:
new_lines = []

for idx, text in tqdm(enumerate(result['content'])):
    dictCount={}
    tag_sen = []
    dictBicount={}
    # try:
    dictCount, tag_sen = build_dictCount(text)
    dictBicount = build_dictBiCount(dictCount, tag_sen)
    graph = build(dictCount, dictBicount)
    rank = pagerank(graph)
    new_text = new_line(rank, dictCount)
    new_lines.append(new_text)
    # except:
    #     print(idx)
    #     break
result['text_rank_content'] = new_lines
result

0it [00:00, ?it/s]

,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000,생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...
11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...
1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...
210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...
1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...
...,...,...,...,...,...
24690,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...
24514,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...
1682,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...
112,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...


In [41]:
result['text_rank_content']

0         생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...
11872     전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...
1918      자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...
210      스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...
1294      트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...
                               ...                        
24690    본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...
24514     바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...
1682     가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...
112      심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...
1122      충치가 더 자주 발생하는 치아가 있을까 정답은 그렇다다. 양치질은 표면의 세균과 ...
Name: text_rank_content, Length: 100, dtype: object

In [57]:
result2 = result.copy()

In [58]:
result2 = result2.reset_index(drop=True)
q_pos = []
for i in tqdm(range(len(result2))):
    q_pos.append(mecab.pos(result2['text_rank_content'][i]))
result2['text_rank_content_token'] = q_pos
result2 = get_df_token(result2, 'text_rank_content', 'text_rank_content_token')
result2

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,text_rank_content_token
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000,생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...,"[생존자, 죄책감, 정신, 질환, 진단, 통계, 편람, 트라우마, 이후, 있, 증상..."
1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,"[전문가, 현장, 있, 목격자, 구조, 인력, 포함, 최대, 명, 트라우마, 심리치..."
2,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...,"[자신, 생존자, 모르, 상담, 치료, 시작, 마음, 회복, 경험, 과정, 적, 알..."
3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상..."
4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,"[트라우마, 치료, 가능, 살, 트라우마, 겪, 있, 사람, 신체, 위협, 가하, ..."
...,...,...,...,...,...,...
95,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ..."
96,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...,"[당뇨병, 당뇨병, 진단, 기준, 과, 같, 당뇨병, 당뇨병, 구별, 중요, 당뇨병..."
97,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,..."
98,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ..."


In [72]:
result3 = jaccard_high(result2, ques_token, 100, 'text_rank_content', 'text_rank_content_token')
result3 = cos_similarity(result3, ques_token, 'text_rank_content', 'text_rank_content_token')
result3 = result3.rename(columns={'jaccard_similarity' : 'tr_jaccard_similarity', 'cosine_similarity' : 'tr_cosine_similarity'})
result3 = result3.reset_index()
result3

,index,text_rank_content,text_rank_content_token,tr_jaccard_similarity,tr_cosine_similarity
0,0,생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...,"[생존자, 죄책감, 정신, 질환, 진단, 통계, 편람, 트라우마, 이후, 있, 증상...",0.322835,0.747027
1,1,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,"[전문가, 현장, 있, 목격자, 구조, 인력, 포함, 최대, 명, 트라우마, 심리치...",0.074830,0.257332
2,17,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,"[이상, 사상자, 이태원, 참사, 트라우마, 정신, 외상, 유족, 생존자, 그날, ...",0.114458,0.201515
3,4,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,"[트라우마, 치료, 가능, 살, 트라우마, 겪, 있, 사람, 신체, 위협, 가하, ...",0.047297,0.191063
4,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.098160,0.188122
...,...,...,...,...,...
95,69,이상 노인 중 이 치매 환자인 셈이다.치매 환자라고 다 똑같은 증상을 보이는 것은...,"[이상, 노인, 치매, 환자, 치매, 환자, 똑같, 증상, 보이, 치매, 환자, 여...",0.021583,0.021091
96,67,기분장애는 이러한 상태를 폭넓게 일컫는 말이다. 기분장애에 속하는 대표 질환이 우...,"[기분, 장애, 상태, 폭넓, 일컫, 말, 기분, 장애, 속하, 대표, 질환, 우울...",0.025806,0.021082
97,92,자신감이 부족하면 수학 성적이 뚝 떨어지고 이로 인해 일상 생활에서도 자신감이 떨...,"[자신, 부족, 수학, 성적, 떨어지, 일상, 생활, 자신, 떨어지, 불안, 있, ...",0.019481,0.018556
98,94,짧은 턱이나 무턱이라면 예쁜 코도 길거나 커보일 수 있습니다. 얼굴에 조화를 맞춘...,"[짧, 턱, 무턱, 코, 길, 있, 얼굴, 조화, 코, 낮, 짧, 코, 높, 예쁘,...",0.014184,0.014765


In [78]:
result5 = result.copy()
result5 = result5.reset_index()
result5 = result5.rename(columns={'index' : 'url_index'})
result5 = result5.reset_index()
result5

,index,url_index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content
0,0,0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000,생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...
1,1,11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...
2,2,1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...
3,3,210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...
4,4,1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...
...,...,...,...,...,...,...,...
95,95,24690,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...
96,96,24514,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...
97,97,1682,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...
98,98,112,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...


In [83]:
result4 = pd.merge(result5, result3[['index', 'tr_jaccard_similarity', 'tr_cosine_similarity']], how='left', on='index')
result4.index = result4['url_index']
del result4['url_index']
result4

,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity
url_index,,,,,,,,
0,0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000,생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and S...,0.322835,0.747027
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332
1918,2,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...,0.087838,0.159616
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122
1294,4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,0.047297,0.191063
...,...,...,...,...,...,...,...,...
24690,95,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,0.046980,0.041911
24514,96,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...,0.048611,0.034031
1682,97,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,0.065476,0.044754


In [86]:
# 과적합 기준이 지금은 1. 어떻게 정할까?
result6 = result4[(result4['jaccard_similarity'] != 1) & (result4['cosine_similarity'] != 1)]
result6

,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity
url_index,,,,,,,,
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332
1918,2,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...,0.087838,0.159616
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122
1294,4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,0.047297,0.191063
15950,5,이태원 참사의 상처는 아직 회복되지 않고 있다. 사회 전반에 트라우마가 자리한 지금...,"[이태원, 참사, 상처, 회복, 사회, 전반, 트라우마, 자리, 삼풍, 백화점, 붕...",0.111392,0.215673,경제 규모나 사회적 시스템으로 봐서 이런 참사가 일어난다는 게 말이 안 되잖아요....,0.070423,0.120011
...,...,...,...,...,...,...,...,...
24690,95,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,0.046980,0.041911
24514,96,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...,0.048611,0.034031
1682,97,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,0.065476,0.044754


In [89]:
for col in ['MainCategory', 'SubCategory', 'WritedAt']:
    for idx in result6.index :
        result6.loc[idx, col] = df.loc[idx, col]
result6

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity,MainCategory,SubCategory,WritedAt
url_index,,,,,,,,,,,
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332,문화,문화생활일반,2022. 11. 3. 14:00
1918,2,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...,0.087838,0.159616,문화,건강,2022. 11. 4. 06:32
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122,문화,건강,2022. 11. 5. 16:23
1294,4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,0.047297,0.191063,문화,건강,2022. 11. 7. 16:36
15950,5,이태원 참사의 상처는 아직 회복되지 않고 있다. 사회 전반에 트라우마가 자리한 지금...,"[이태원, 참사, 상처, 회복, 사회, 전반, 트라우마, 자리, 삼풍, 백화점, 붕...",0.111392,0.215673,경제 규모나 사회적 시스템으로 봐서 이런 참사가 일어난다는 게 말이 안 되잖아요....,0.070423,0.120011,문화,문화생활일반,2022. 11. 28. 08:20
...,...,...,...,...,...,...,...,...,...,...,...
24690,95,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,0.046980,0.041911,문화,건강,2022. 11. 16. 12:42
24514,96,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,바로 당뇨병이다.당뇨병 진단 기준은 과 이 같다. 그러나 당뇨병과 당뇨병을 구별하...,0.048611,0.034031,문화,건강,2022. 11. 19. 11:33
1682,97,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,0.065476,0.044754,문화,건강,2022. 11. 2. 11:31


In [99]:
target_column = df.iloc[0, :]
target_column

Content         기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 , ,“내가 조금만 다르게 행...
DomainID                                                        0
MainCategory                                                   문화
PhotoURL        https://img2.daumcdn.net/thumb/R658x0.q70/?fna...
Press                                                        중앙일보
Stickers        {'article_id': '20221107000430780', '좋아요': 0, ...
SubCategory                                                    건강
Title                              살아가는 사람들의 고통 곁에 선다는 것 [건강한 가족]
URL                        https://v.daum.net/v/20221107000430780
WritedAt                                       2022. 11. 7. 10:08
Writer                                                        입력 
Name: 0, dtype: object

In [141]:
# total_score
# 유사도 스캐일링
content_ja_scaled = (result6['jaccard_similarity'] - result6['jaccard_similarity'].min()) /( result6['jaccard_similarity'].max() - result6['jaccard_similarity'].min())
content_co_scaled = (result6['cosine_similarity'] - result6['jaccard_similarity'].min())/( result6['cosine_similarity'].max() - result6['cosine_similarity'].min())
cotent_score = (1/3 * 50) * content_ja_scaled + (2/3 * 50) * content_co_scaled

# 본문 핵심문장
tr_content_ja_scaled = result6['tr_jaccard_similarity'] / (result6['tr_jaccard_similarity'].max() -  result6['tr_jaccard_similarity'].min())
tr_content_co_scaled = result6['tr_cosine_similarity'] / (result6['tr_cosine_similarity'].max() -  result6['tr_jaccard_similarity'].min())
tr_cotent_score = (1/3 * 30) * tr_content_ja_scaled + (2/3 * 30) * tr_content_co_scaled

# 카테고리
main_cate_score = np.where(result6['MainCategory'] == target_column['MainCategory'], 1, 0) * 0.1
sub_cate_score = np.where(result6['SubCategory'] == target_column['SubCategory'], 1, 0) * 0.05

# 날짜
result_date = pd.to_datetime(result6['WritedAt']).dt.day.values
target_date = int(pd.to_datetime(target_column['WritedAt']).day)
date_score = np.where(abs(result_date - target_date) <= 1, 1, 
                      np.where(abs(result_date - target_date) <= 3, 0.5, 
                                np.where(abs(result_date - target_date) <= 7, 0.3, 0))) * 0.05

total_score = cotent_score + tr_cotent_score + main_cate_score + sub_cate_score + date_score
result6['total_score'] = total_score
result6 = result6.sort_values(by='total_score', ascending=False)
result6['total_score'] = result6['total_score'].apply(lambda x : round(x,2))
result6

,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity,MainCategory,SubCategory,WritedAt,total_score
url_index,,,,,,,,,,,,
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332,문화,문화생활일반,2022. 11. 3. 14:00,64.57
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122,문화,건강,2022. 11. 5. 16:23,51.80
304,6,요즘 우울한 소식들로 불안감과 우울증 트라우마Trauma심리적 외상를 호소하는 사람...,"[우울, 소식, 불안감, 우울증, 트라우마, 심리, 외상, 호소, 사람, 늘, 국민...",0.146998,0.211584,국민의 정신건강이 위협받고 있다.육체적으로 특별한 이상이 없어도 정신적 또는 사회적...,0.103448,0.140279,문화,건강,2022. 11. 15. 18:45,50.46
1294,4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,0.047297,0.191063,문화,건강,2022. 11. 7. 16:36,46.89
11875,17,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,"[이상, 사상자, 이태원, 참사, 트라우마, 정신, 외상, 유족, 생존자, 그날, ...",0.136187,0.166265,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,0.114458,0.201515,문화,문화생활일반,2022. 11. 3. 14:13,46.20
...,...,...,...,...,...,...,...,...,...,...,...,...
18545,83,마치 강간당한 기분이었어. 언젠가의 술자리에서 지인이 말했다. 그는 과거에 경험한 ...,"[강간, 기분, 술, 자리, 지인, 말, 과거, 경험, 부당, 일, 떠올리, 분개,...",0.104839,0.062710,마치 강간당한 기분이었어. 그것은 정확하지 않은 언어 때문이었다.지인은 부당한 일을...,0.007194,0.023454,문화,책,2022. 11. 25. 09:06,-2.27
23808,92,초등학생 고교생 수포자사소한 성공노력도 격려칭찬해야수학 과목을 유독 두려워하는 사람...,"[초등학생, 고교, 수포, 성공, 노력, 격려, 칭찬, 수학, 과목, 하, 사람, ...",0.108179,0.045867,자신감이 부족하면 수학 성적이 뚝 떨어지고 이로 인해 일상 생활에서도 자신감이 떨...,0.019481,0.018556,문화,건강,2022. 11. 24. 17:10,-2.51
1122,99,치아 모양에 따라 달라질 수도충치치아우식증는 많은 이들의 고민이다. 치과를 무서워하...,"[치아, 모양, 충치, 치아우식증, 많, 고민, 치과, 무서워하, 사람, 충치, 드...",0.107884,0.032391,충치가 더 자주 발생하는 치아가 있을까 정답은 그렇다다. 양치질은 표면의 세균과 ...,0.032468,0.025622,문화,건강,2022. 11. 10. 19:03,-2.72


In [138]:
df.iloc[0:1, :]

,Content,DomainID,MainCategory,PhotoURL,Press,Stickers,SubCategory,Title,URL,WritedAt,Writer
0,"기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 , ,“내가 조금만 다르게 행...",0,문화,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,중앙일보,"{'article_id': '20221107000430780', '좋아요': 0, ...",건강,살아가는 사람들의 고통 곁에 선다는 것 [건강한 가족],https://v.daum.net/v/20221107000430780,2022. 11. 7. 10:08,입력


In [142]:
result6.head()

,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity,MainCategory,SubCategory,WritedAt,total_score
url_index,,,,,,,,,,,,
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332,문화,문화생활일반,2022. 11. 3. 14:00,64.57
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122,문화,건강,2022. 11. 5. 16:23,51.80
304,6,요즘 우울한 소식들로 불안감과 우울증 트라우마Trauma심리적 외상를 호소하는 사람...,"[우울, 소식, 불안감, 우울증, 트라우마, 심리, 외상, 호소, 사람, 늘, 국민...",0.146998,0.211584,국민의 정신건강이 위협받고 있다.육체적으로 특별한 이상이 없어도 정신적 또는 사회적...,0.103448,0.140279,문화,건강,2022. 11. 15. 18:45,50.46
1294,4,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...,0.047297,0.191063,문화,건강,2022. 11. 7. 16:36,46.89
11875,17,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,"[이상, 사상자, 이태원, 참사, 트라우마, 정신, 외상, 유족, 생존자, 그날, ...",0.136187,0.166265,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,0.114458,0.201515,문화,문화생활일반,2022. 11. 3. 14:13,46.20


In [132]:
result6.head()

,index,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content,tr_jaccard_similarity,tr_cosine_similarity,MainCategory,SubCategory,WritedAt,total_score
url_index,,,,,,,,,,,,
11872,1,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...,0.074830,0.257332,문화,문화생활일반,2022. 11. 3. 14:00,69.88
304,6,요즘 우울한 소식들로 불안감과 우울증 트라우마Trauma심리적 외상를 호소하는 사람...,"[우울, 소식, 불안감, 우울증, 트라우마, 심리, 외상, 호소, 사람, 늘, 국민...",0.146998,0.211584,국민의 정신건강이 위협받고 있다.육체적으로 특별한 이상이 없어도 정신적 또는 사회적...,0.103448,0.140279,문화,건강,2022. 11. 15. 18:45,67.06
210,3,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,0.098160,0.188122,문화,건강,2022. 11. 5. 16:23,66.69
11875,17,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,"[이상, 사상자, 이태원, 참사, 트라우마, 정신, 외상, 유족, 생존자, 그날, ...",0.136187,0.166265,이상의 사상자를 낸 이태원 참사의 트라우마정신적 외상가 유족과 생존자는 물론 그날...,0.114458,0.201515,문화,문화생활일반,2022. 11. 3. 14:13,66.04
1885,29,직접 경험처럼 여기는 뇌 속 거울 뉴런으로 인해 정신적 고통 받아특정 시간대 스마트...,"[경험, 뇌, 속, 거울, 뉴런, 정신, 고통, 받, 특정, 시간, 스마트폰, 하,...",0.141129,0.137253,직접 경험처럼 여기는 뇌 속 거울 뉴런으로 인해 정신적 고통 받아특정 시간대 스마트...,0.129730,0.137254,문화,건강,2022. 11. 3. 11:14,65.40


In [135]:
result.head(10).iloc[1:6,:]

,content,content_token,jaccard_similarity,cosine_similarity,text_rank_content
11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,전문가들은 현장에 있던 목격자와 구조인력 등까지 포함하면 최대 명에 대한 트라우마...
1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,자신이 생존자인지 모르겠다던 그는 상담치료를 시작해 마음의 회복을 경험한 과정을 ...
210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...
1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,트라우마는 치료가 가능할까 누구나 살면서 트라우마를 겪을 수 있다. 다른 사람에게...
15950,이태원 참사의 상처는 아직 회복되지 않고 있다. 사회 전반에 트라우마가 자리한 지금...,"[이태원, 참사, 상처, 회복, 사회, 전반, 트라우마, 자리, 삼풍, 백화점, 붕...",0.111392,0.215673,경제 규모나 사회적 시스템으로 봐서 이런 참사가 일어난다는 게 말이 안 되잖아요....


In [87]:
df.columns

Index(['Content', 'DomainID', 'MainCategory', 'PhotoURL', 'Press', 'Stickers',
       'SubCategory', 'Title', 'URL', 'WritedAt', 'Writer'],
      dtype='object')

In [102]:
import numpy as np
target_column['MainCategory']
main_cate_score = np.where(result6['MainCategory'] == target_column['MainCategory'], 1, 0)
sub_cate_score = np.where(result6['SubCategory'] == target_column['SubCategory'], 1, 0)


In [125]:
result_date = pd.to_datetime(result6['WritedAt']).dt.day.values
target_date = int(pd.to_datetime(target_column['WritedAt']).day)
date_score = np.where(abs(result_date - target_date) <= 1, 1, 
                      np.where(abs(result_date - target_date) <= 3, 0.5, 
                                np.where(abs(result_date - target_date) <= 7, 0.3, 0)))
date_score

array([0.3, 0.5, 0.5, 1. , 0. , 0. , 0. , 0.5, 0.5, 0.3, 1. , 0. , 0.3,
       0.5, 0.5, 0. , 0.3, 0. , 0.5, 0.3, 0.5, 0. , 0.3, 1. , 0.5, 0. ,
       0.3, 0.3, 0.3, 1. , 0.3, 0.3, 0.5, 0.5, 0. , 0.3, 0.3, 0.5, 0.5,
       0. , 0.5, 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ,
       1. , 0. , 0. , 0.5, 0.5, 0.3, 0.3, 0.5, 0. , 0.5, 1. , 0.3, 0.3,
       0. , 0.3, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3,
       0. , 0. , 1. , 0. , 0. , 0. , 1. , 0.3, 0. , 0.3, 0. , 1. , 1. ,
       0. , 0.3, 0.3, 0. , 0. , 0.3, 0. , 0.5])

In [118]:
pd.to_datetime(result6['WritedAt']).dt.day.values

array([ 3.,  4.,  5.,  7., 28., 15., 15.,  4.,  4.,  2.,  8., 23.,  3.,
        4.,  4., nan,  3., 16.,  5.,  3.,  5., nan,  3.,  8.,  4., 23.,
        2.,  2.,  3.,  7., 11., 13.,  5.,  4., 25.,  3.,  3., 10.,  4.,
       17.,  5., 20.,  2., 24., 17., 27., 26., 22., 18., 27.,  5., 30.,
        7., 23., 22., 10.,  4.,  2., 14.,  4., 15.,  5.,  6.,  2., 14.,
       24., 11.,  5., 18., 17., 24., 17., nan, 16., 29., 26., 16., 14.,
       16., 30.,  8., 19., 25., 26.,  7.,  1., 30.,  1., 17.,  7.,  6.,
       24., 11.,  1., 16., 19.,  2., 15., 10.])

In [123]:
int(pd.to_datetime(target_column['WritedAt']).day)

7

# 파생변수 생성
1. 본문 유사도 50% : 자카드 / 코사인 비율 1:2. 
-> 유사도의 경우 가장 근접한 수치 기준으로 MinMaxScaling
-> 유사도의 비중을 더욱 높게 반영하고 연산속도를 위해 유사도 기준 상위 100개에 
2. 본문 핵심문장 유사도 30% : 본문과 산정방식 동일. / TextRank 같은 걸로 한 
api 서버에서 5초 도는동안 사용자 이탈 가능성 그만큼 높아짐 (TextRank 100개 기준 약 5초)
TextRank 결과값이 거의 제목과 비슷하면 굳이 쓸 필요 없을 수도?
-> 잼이 발생하고.
-> db 전체 
전체 텍스트랭크 돌리면 10만개 기준 13시간 정도

1~2문장정도 요약해서 유사도를 해볼까.
3. 같은 메인카테고리 주제에 대해선 10% 가중치
4. 같은 서브카테고리 주제에 대해선 5% 가중치
5. 1일 근접한 기사에 대해서는 1, 3일내 0.5, 7일내 0.3, 이외의 기사 가중치 0, NaN값은 0으로 처리. 전체 가중치 지분 5% 정도
<!-- 6. 같은 언론사의 기사에 대해서는 5% 가중치 -->

## 함수화

In [18]:
result

,content,content_token,jaccard_similarity,cosine_similarity
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000
11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618
1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944
210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380
1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702
...,...,...,...,...
24690,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319
24514,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821
1682,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516
112,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286


In [ ]:
def total_score(result_df):
    for col in ['Title', 'MainCategory', 'SubCategory', 'WritedAt']:
        for idx in result_df.index :
            result_df.loc[idx, col] = df.loc[idx, col]
    result_df


In [48]:
result2 = result.copy()

# 뉴스 인덱스 처리
result2 = result2.reset_index()
result2 = result2.rename(columns={'index' : 'url_index'})
result2 = result2.reset_index()
result2
for col in ['Title', 'MainCategory', 'SubCategory', 'WritedAt']:
        re_idx = 0
        for idx in result2['url_index'] :
            result2.loc[re_idx, col] = df.loc[idx, col]
            print(cnt)
            cnt += 1
result2

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349


,index,url_index,content,content_token,jaccard_similarity,cosine_similarity,Title,MainCategory,SubCategory,WritedAt
0,0,0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000,"충치 잘 생기는 이, 따로 있다?",문화,건강,2022. 11. 10. 19:03
1,1,11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618,NaN,NaN,NaN,NaN
2,2,1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944,NaN,NaN,NaN,NaN
3,3,210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380,NaN,NaN,NaN,NaN
4,4,1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,95,24690,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319,NaN,NaN,NaN,NaN
96,96,24514,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821,NaN,NaN,NaN,NaN
97,97,1682,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516,NaN,NaN,NaN,NaN
98,98,112,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286,NaN,NaN,NaN,NaN


In [44]:
result10 = result.copy()
result10 = result10.drop(0)
df.loc[result10['jaccard_similarity'].idxmax(), :]

Unnamed: 0                                                  51729
Content         먼저 송대윤 의원(더불어민주당‧유성구2)은 오전 감사에서 온통대전 질의에 집중했다....
DomainID                                                        0
MainCategory                                                   사회
PhotoURL        https://img3.daumcdn.net/thumb/R658x0.q70/?fna...
Press                                                         더팩트
Stickers        {'article_id': '20221111155552224', '좋아요': 0, ...
SubCategory                                                    전국
Title                [대전 행감] '온통대전' 폐지 공방..."실효성 없어" vs "소상공인 도움"
URL                        https://v.daum.net/v/20221111155552224
WritedAt                                      2022. 11. 11. 15:55
Writer                                                        라안일
Name: 47858, dtype: object

In [45]:
df.loc[result10['cosine_similarity'].idxmax(), :]

Unnamed: 0                                                    147
Content          [이데일리 박지혜 기자] 흉기로 저항하던 50대 남성에 실탄을 쏴 검거한 경찰 대...
DomainID                                                        0
MainCategory                                                   사회
PhotoURL        https://img4.daumcdn.net/thumb/R658x0.q70/?fna...
Press                                                        이데일리
Stickers        {'article_id': '20221121101256601', '좋아요': 130...
SubCategory                                                 사건/사고
Title                      "사람 보이면 다 죽인다"...'코드0' 경찰, 실탄까지 쏜 상황보니
URL                        https://v.daum.net/v/20221121101256601
WritedAt                                      2022. 11. 21. 10:12
Writer                                                        박지혜
Name: 124, dtype: object

In [38]:
result2 = result.copy()

# 뉴스 인덱스 처리
result2 = result2.reset_index()
result2 = result2.rename(columns={'index' : 'url_index'})
result2 = result2.reset_index()

# 동일기사 검증 임계치
threshold = 1
result2 = result2[(result2['jaccard_similarity'] < threshold) & (result2['cosine_similarity'] < threshold)]

# 필요한 컬럼 가져오기
for col in ['MainCategory', 'SubCategory', 'WritedAt']:
    for idx in result2.index :
        result2.loc[idx, col] = df.loc[result2.loc[idx, 'url_index'], col]
# # 타이틀 토큰화
# q_pos = []
# for idx in tqdm(result2.index):
#     q_pos.append(mecab.pos(result2.loc[idx,'Title']))
# result2['content__token'] = q_pos
# result2 = result2.reset_index()
# display(result2)

# 타겟 타이틀 불러오기
target_column = df.iloc[0:1, :]
target_title = target_column['Title'].values[0]
# 타이틀 토큰된거로 유사도 계산

ques_token_title = get_ques_token(target_title)
ques_token_title

result3 = result2.copy()
result3 = jaccard_high(result3, ques_token_title, 100, 'content', 'content_token')
result3 = cos_similarity(result3, ques_token_title, 'content', 'content_token')
result3 = result3.rename(columns={'jaccard_similarity' : 'Ti_jaccard_similarity', 'cosine_similarity' : 'Ti_cosine_similarity'})
result3 = result3.reset_index()

# 타이틀 토큰화 결과 합치기
result2 = pd.merge(result2, result3[['index', 'Ti_jaccard_similarity', 'Ti_cosine_similarity']], how='left', on='index')
# result2.index = result2['url_index']
del result2['index']

# total_score
# 유사도 스캐일링
content_ja_scaled = (result2['jaccard_similarity'] - result2['jaccard_similarity'].min()) /( result2['jaccard_similarity'].max() - result2['jaccard_similarity'].min())
content_co_scaled = (result2['cosine_similarity'] - result2['cosine_similarity'].min())/( result2['cosine_similarity'].max() - result2['cosine_similarity'].min())
cotent_score = (1/3 * 50) * content_ja_scaled + (2/3 * 50) * content_co_scaled

# 본문 핵심문장
tr_content_ja_scaled = (result2['Ti_jaccard_similarity'] -  result2['Ti_jaccard_similarity'].min() )/ (result2['Ti_jaccard_similarity'].max() -  result2['Ti_jaccard_similarity'].min())
tr_content_co_scaled = (result2['Ti_cosine_similarity'] -  result2['Ti_cosine_similarity'].min())/ (result2['Ti_cosine_similarity'].max() -  result2['Ti_cosine_similarity'].min())
tr_cotent_score = (1/3 * 30) * tr_content_ja_scaled + (2/3 * 30) * tr_content_co_scaled

# 카테고리
main_cate_score = np.where(result2['MainCategory'] == target_column['MainCategory'].values[0], 1, 0) * 0.1
sub_cate_score = np.where(result2['SubCategory'] == target_column['SubCategory'].values[0], 1, 0) * 0.05

# 날짜
result_date = pd.to_datetime(result2['WritedAt']).dt.day.values
target_date = int(pd.to_datetime(target_column['WritedAt'].values[0]).day)
date_score = np.where(abs(result_date - target_date) <= 1, 1, 
                      np.where(abs(result_date - target_date) <= 3, 0.5, 
                                np.where(abs(result_date - target_date) <= 7, 0.3, 0))) * 0.05

total_score = cotent_score + tr_cotent_score + main_cate_score + sub_cate_score + date_score
result2['total_score'] = total_score
result2 = result2.sort_values(by='total_score', ascending=False)
result2['total_score'] = result2['total_score'].apply(lambda x : round(x,2))
result2



,url_index,content,content_token,jaccard_similarity,cosine_similarity,MainCategory,SubCategory,WritedAt,Ti_jaccard_similarity,Ti_cosine_similarity,total_score
2,12820,국방위 원만 예산 반영대전시 국토 균형발전 저해 우려예결위 소위서 복구 요청할 것방...,"[국방위, 원만, 예산, 반영, 대전시, 국토, 균형발전, 저해, 우려, 예결, 위...",0.125954,0.169677,사회,지역일반,2022. 11. 7. 01:02,0.019108,0.104253,63.36
0,124,흉기로 저항하던 남성에 실탄을 쏴 검거한 경찰 대응에 대해 전문가는 적절했다...,"[흉기, 저항, 남성, 실탄, 검거, 경찰, 대응, 전문가, 평가, 부산, 사상, ...",0.112628,0.204043,사회,사건/사고,2022. 11. 21. 10:12,0.010582,0.131455,63.07
1,31453,국방위원회 전체소위 회의록으로 살펴 본 이전 예산 삭감 과정국방위 민주당 의원 반대...,"[국방위원회, 전체, 소위, 회의록, 이전, 예산, 삭감, 과정, 국방위, 민주당,...",0.119792,0.182006,사회,충청,2022. 11. 9. 19:20,0.010274,0.118542,61.30
7,32957,이장우 시장 대통령과 전화통화 이외 무엇을 했나대전 더불어민주당 대전시당은 국...,"[이장우, 시장, 대통령, 전화, 통화, 이외, 대전, 더불, 민주당, 대전, 시당...",0.122727,0.125209,사회,충청,2022. 11. 4. 15:01,0.027523,0.105700,56.14
3,36776,지자체 지역 정치권 언론 등 한 목소리 낸 끝에 약 만에 복원예결특위 국방위 동의 ...,"[지자체, 지역, 정치, 언론, 목소리, 끝, 약, 복원, 예결, 특위, 국방위, ...",0.116041,0.149446,사회,충청,2022. 11. 20. 19:03,0.021277,0.111968,55.51
...,...,...,...,...,...,...,...,...,...,...,...
98,12927,우성우 행정안전부 지진방재정책과장 주최로 대구시 동구 메리어트호텔에서 열린 제 재...,"[우성우, 행정, 지진, 방재, 정책, 과장, 주최, 대구시, 동구, 메리어, 트,...",0.105042,0.020785,사회,지역일반,2022. 11. 10. 18:14,0.007874,0.001612,4.50
89,50230,공청회 이어 행안위 법안심사 소위 통과다음 달 초 법사위 통과되면 본회의 처리연내...,"[공청회, 행안, 위, 법안, 심사, 소위, 통과, 다음, 달, 법, 통과, 본회,...",0.104348,0.037564,사회,서울,2022. 11. 28. 14:22,0.000000,0.000000,4.47
77,45642,이달 말 예정 타당성 발표 연기응급 의료 공백 해소 대책 차질광주지역 최대 현안이자...,"[이달, 말, 예정, 발표, 연기, 응급, 의료, 공백, 해소, 대책, 차질, 광주...",0.102679,0.042417,사회,서울,2022. 11. 17. 05:02,0.000000,0.000000,4.31
94,9827,병무청장 심각하게 생각하고 있다서울 일부 대학교 학생들이 예비군 훈련 참석으로 ...,"[병무, 청장, 생각, 서울, 일부, 대학교, 학생, 예비군, 훈련, 참석, 이익,...",0.103321,0.028164,사회,지역일반,2022. 11. 15. 14:14,0.006135,0.001686,4.27


In [35]:
target_column

,Unnamed: 0,Content,DomainID,MainCategory,PhotoURL,Press,Stickers,SubCategory,Title,URL,WritedAt,Writer
0,0,"국회 행안위 소위 25억 삭감,전자충격기, 순찰차 유류비도 줄줄이 깎여[아시아경제 ...",0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,아시아경제,"{'article_id': '20221130104657051', '좋아요': 133...",사건/사고,尹대통령 주문한 ‘1인 1총기’ 예산 25억·전자충격기 전액 삭감,https://v.daum.net/v/20221130104657051,2022. 11. 30. 10:46,장세희
